あらかじめGoogle Driveに'Colab Notebooks/Kuzushiji/Kuzushiji-49'のディレクトリ構造を作っておく。

Kuzushiji-49
*   k49_classmap.csv
*   k49-train-imgs.npz
*   k49-train-labels.npz
*   k49-test-imgs.npz
*   k49-test-labels.npz

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
main_path = 'drive/My Drive/Colab Notebooks/Kuzushiji/Kuzushiji-49'

In [0]:
import os
import zipfile
import numpy as np
import pandas as pd
import glob

In [0]:
import keras
from keras.utils import np_utils
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Concatenate, AveragePooling2D, GlobalAveragePooling2D, Dense
from keras.callbacks import ModelCheckpoint, CSVLogger

Using TensorFlow backend.


In [0]:
n_class = 49
img_shape = (28, 28, 1)

In [0]:
train_X_file = np.load(os.path.join(main_path, 'k49-train-imgs.npz'))
train_Y_file = np.load(os.path.join(main_path, 'k49-train-labels.npz'))

In [0]:
train_X = train_X_file['arr_0']
train_Y = train_Y_file['arr_0']

In [0]:
train_X = np.expand_dims(train_X, axis = 3) / 255.0
train_X.shape

(232365, 28, 28, 1)

In [0]:
train_Y.shape

(232365,)

In [0]:
train_Y = np_utils.to_categorical(train_Y, n_class)
train_Y.shape

(232365, 49)

CNNを構築

In [0]:
def cba(inputs, filters, kernel_size, strides):
  x = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = 'same')(inputs)
  x = BatchNormalization()(x)
  outputs = Activation('relu')(x)
  
  return outputs

In [0]:
inputs = Input(shape = img_shape)
    
x = cba(inputs, filters = 64, kernel_size = (2, 2), strides = (1, 1))
x = cba(x, filters = 64, kernel_size = (3, 3), strides = (2, 2))
x = cba(x, filters = 128, kernel_size = (3, 3), strides = (2, 2))
x = cba(x, filters = 256, kernel_size = (3, 3), strides = (2, 2))
        
x = GlobalAveragePooling2D()(x)
            
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
            
x = Dense(n_class)(x)
            
outputs = Activation('softmax')(x)
            
model = Model(inputs, outputs)

optimizer = keras.optimizers.Adam()

model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
acti

In [0]:
save_path = os.path.join(main_path, 'model')
CNN_name = 'CNN_model_2'

In [0]:
if not os.path.exists(save_path):
  os.mkdir(save_path)
        
if not os.path.exists(os.path.join(save_path, CNN_name)):
  os.mkdir(os.path.join(save_path, CNN_name))

In [0]:
checkpoint = ModelCheckpoint(os.path.join(save_path, CNN_name, CNN_name + '_{epoch:d}.h5'))
csv_logger = CSVLogger(os.path.join(save_path, CNN_name + '.csv'), append = True)

In [0]:
model.fit(train_X, train_Y, epochs = 100, callbacks = [checkpoint, csv_logger], validation_split = 0.1)